<a href="https://colab.research.google.com/github/rashaKM2025/Data-Science-Salary-Prediction/blob/main/tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/glassdoor/glassdoorjobs_normalization.csv')
df

,unnamed:_0,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,...,type_of_ownership_encoded,industry_encoded,sector_encoded,salary_range,city,state,company_size_encoded,revenue_million,company_age,job_title_simplified
0,0,data scientist,$53K-$91K (Glassdoor est.),Job Description,3.8,tecolote research\n3.8,"albuquerque, nm","goleta, ca",501 to 1000 employees,1973,...,2,3,2,NaN,albuquerque,nm,4.0,75.0,53,scientist
1,1,healthcare data scientist,$63K-$112K (Glassdoor est.),Job Description,3.4,university of maryland medical system\n3.4,"linthicum, md","baltimore, md",10000+ employees,1984,...,8,28,12,NaN,linthicum,md,7.0,3500.0,42,scientist
2,2,data scientist,$80K-$90K (Glassdoor est.),Job Description,4.8,knowbe4\n4.8,"clearwater, fl","clearwater, fl",501 to 1000 employees,2010,...,2,48,6,NaN,clearwater,fl,4.0,300.0,16,scientist
3,3,data scientist,$56K-$97K (Glassdoor est.),Job Description,3.8,pnnl\n3.8,"richland, wa","richland, wa",1001 to 5000 employees,1965,...,5,18,19,NaN,richland,wa,5.0,750.0,61,scientist
4,4,data scientist,$86K-$143K (Glassdoor est.),Job Description,2.9,affinity solutions\n2.9,"new york, ny","new york, ny",51 to 200 employees,1998,...,2,2,6,NaN,new york,ny,2.0,0.0,28,scientist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,922,data scientist,-1,Job Description,4.7,"steel point solutions, llc\n4.7","alexandria, va","calverton, md",1 to 50 employees,2013,...,8,19,13,NaN,alexandria,va,1.0,0.0,13,scientist
593,925,data scientists,-1,Job Description,3.3,miracle software systems\n3.3,michigan,"novi, mi",51 to 200 employees,1994,...,2,35,13,NaN,michigan,unknown,2.0,75.0,32,scientist
594,928,"associate scientist / sr. associate scientist,...",$59K-$125K (Glassdoor est.),Job Description,4.0,23andme\n4.0,"south san francisco, ca","sunnyvale, ca",501 to 1000 employees,2006,...,2,8,5,NaN,south san francisco,ca,4.0,0.0,20,scientist
595,945,machine learning engineer (nlp),$80K-$142K (Glassdoor est.),Job Description,4.1,ck-12 foundation\n4.1,"palo alto, ca","palo alto, ca",1 to 50 employees,2007,...,2,36,9,NaN,palo alto,ca,1.0,0.0,19,engineer


In [35]:
df = df.drop(columns=["unnamed: 0"])


In [36]:
df.columns


Index(['job title', 'salary estimate', 'job description', 'rating', 'company name', 'location', 'headquarters', 'size', 'founded', 'type of ownership', 'industry', 'sector', 'revenue', 'competitors', 'min salary', 'max salary', 'company name clean', 'avg salary', 'min salary norm', 'max salary norm', 'avg salary norm', 'type of ownership encoded', 'industry encoded', 'sector encoded', 'salary range', 'city', 'state', 'company size encoded', 'revenue million', 'company age', 'job title simplified'], dtype='object')

In [37]:
df.columns = df.columns.str.replace("_", " ")

In [38]:
def describe_structure(df):
    summary = []
    for col in df.columns:
        dtype = df[col].dtype
        col_type = "Numeric" if np.issubdtype(dtype, np.number) else "Categorical"
        measurement = "Ratio" if col_type == "Numeric" else "Nominal"
        units = "Years" if "age" in col else "Dollars" if "salary" in col else "-"
        unique_vals = df[col].nunique()
        null_vals = df[col].isnull().sum()
        min_val = df[col].min() if col_type == "Numeric" else "-"
        max_val = df[col].max() if col_type == "Numeric" else "-"
        outliers = "Yes" if col_type == "Numeric" and df[col].std() > 2 * df[col].mean() else "No"
        summary.append([
            col, col_type, dtype, measurement, units,
            f"{min_val}-{max_val}", min_val, max_val,
            unique_vals, null_vals, outliers
        ])
    return pd.DataFrame(summary, columns=[
        "Variable", "Types of Data", "Data Types", "Measurement Level",
        "Units", "Range", "Min Value", "Top Value",
        "Unique Values", "Null Values", "Outliers"
    ])



In [39]:
structure_table = describe_structure(df)


In [40]:
df.shape

(597, 31)

In [41]:
print(structure_table.head(10))

            Variable Types of Data Data Types Measurement Level    Units    Range Min Value Top Value  Unique Values  Null Values Outliers
0          job title   Categorical     object           Nominal        -      ---         -         -            328            0       No
1    salary estimate   Categorical     object           Nominal  Dollars      ---         -         -            415            0       No
2    job description   Categorical     object           Nominal        -      ---         -         -              1            0       No
3             rating       Numeric    float64             Ratio        -  1.9-5.0       1.9       5.0             31           21       No
4       company name   Categorical     object           Nominal        -      ---         -         -            448            0       No
5           location   Categorical     object           Nominal        -      ---         -         -            236            0       No
6       headquarters   Cate

In [42]:
def describe_statistics(df):
    stats = []
    for col in df.select_dtypes(include=[np.number]).columns:
        freq = "-"  # optional for numeric
        percentiles = np.percentile(df[col].dropna(), [25, 50, 75])
        completeness = f"{100 * df[col].notnull().mean():.1f}%"
        mean = df[col].mean()
        median = df[col].median()
        mode = df[col].mode().iloc[0] if not df[col].mode().empty else "-"
        std = df[col].std()
        var = df[col].var()
        skew = df[col].skew()
        kurt = df[col].kurtosis()
        stats.append([col, freq, percentiles, completeness, mean, median, mode, std, var, skew, kurt])
    return pd.DataFrame(stats, columns=["Variable", "Frequency", "Percentile (25th, 50th, 75th)", "Data Completeness", "Mean", "Median", "Mode", "Standard Deviation", "Variance", "Skewness", "Kurtosis"])


In [43]:
stats_table = describe_statistics(df)


In [44]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [45]:
print(stats_table.head(10))

                    Variable Frequency Percentile (25th, 50th, 75th) Data Completeness         Mean   Median  Mode  Standard Deviation       Variance  Skewness   Kurtosis
0                     rating         -               [3.4, 3.8, 4.2]             96.5%     3.799826     3.80  3.80            0.610303       0.372470 -0.243933   0.006523
1                    founded         -      [1939.0, 1993.0, 2008.0]            100.0%  1771.030151  1993.00 -1.00          605.593504  366743.492378 -2.574669   4.703294
2                 min salary         -             [0.1, 0.12, 0.12]              1.5%     0.122778     0.12  0.12            0.034197       0.001169  1.611373   3.010625
3                 max salary         -          [0.135, 0.14, 0.145]              1.5%     0.148333     0.14  0.14            0.042647       0.001819  1.749134   5.031299
4                 avg salary         -          [0.12, 0.13, 0.1325]              1.5%     0.135556     0.13  0.12            0.037992       0.00

In [46]:
structure_table.to_excel("structure_table.xlsx", index=False)
stats_table.to_excel("statistics_table.xlsx", index=False)

In [19]:
with pd.ExcelWriter("glassdoor_tables.xlsx") as writer:
    structure_table.to_excel(writer, sheet_name="structure", index=False)
    stats_table.to_excel(writer, sheet_name="statistics", index=False)

In [22]:
from google.colab import files
files.download("glassdoor_tables.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>